# Write-up and code for Jan 30

## To Do
- Implement Black-Scholes formulas for European Call/Put Pricing
- Implement standard binary tree/grid-based numerical algorithm for American Option Pricing and ensure it validates against Black-Scholes formula for Europeans
- Implement Longstaff-Schwartz Algorithm and ensure it validates against binary tree/grid-based solution for path-independent options
- Explore/Discuss an Approximate Dynamic Programming solution as an alternative to Longstaff-Schwartz Algorithm

## Black-Scholes formulas
According the the Black-Scholes Equation the price of an option is described by
$$ 
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2S^2\frac{\partial^2V}{\partial S^2} + rS\frac{\partial V}{\partial S} - rV = 0
$$
Where
- $S(t)$ is the price of the underlying asset
- $V(S,t)$ is the price of the option
- $\sigma$ is the standard deviation of the return for the underlying asset
- $r$ is the risk-free interest rate

Solving this equation for a European Call Option gives us
$$
\begin{align}
C(S_t, t) & = N(d_1) S_t - N(d_2) PV(K)\\
d_1 & = \frac{1}{\sigma\sqrt{T-t}}\Big[ ln\big( \frac{S_t}{K}\big) + \big( r + \frac{\sigma^2}{2}\big)(T-t)\big) \Big]\\
d_2 & = d_1 - \sigma\sqrt{T-t}\\
PV(K) & = Ke^{-r(T-t)}
\end{align}
$$

Using the put-call parity then gives us the price of a European Put Option

$$
\begin{align}
P(S_t,t) & = Ke^{-r(T-t)} - S_t + C(S_t,t)\\
& = N(-d_2)Ke^{-r(T-t)} - N(-d1)S_t
\end{align}
$$

Where
- $N(\cdot)$ is the cdf for a standard normal distribution
- $K$ is the strike price
- $T-t$ is the time left to maturity

The following code finds the price using the equations above:

In [7]:
from typing import NamedTuple

class EuropeanOption(NamedTuple):
    call: bool # true if call option, false if put
    S: float # underlying asset price
    K: float # strike price
    sigma: float # standard deviation of the return for the stock price
    tau: float # time to maturity (T-t), in years
    r: float # annual interest rate

In [18]:
import numpy as np
from scipy.stats import norm

def Black_Scholes_Price_Call(option: EuropeanOption) -> float:
    # function to calculate the Black-Scholes price for a European Option
    if option.call:
        # return the price of a european call option
        return norm.cdf(d1(option))*option.S - norm.cdf(d2(option))*PV(option)*option.K
    
    return norm.cdf(-d2(option))*PV(option)*option.K - norm.cdf(-d1(option))*option.S


def d1(option: EuropeanOption) -> float:
    # function to calculate the intermediate value d1 in the Black-Scholes formula
    return 1/(option.sigma * np.sqrt(option.tau))*(np.log(option.S/option.K) + (option.r + option.sigma**2/2)*option.tau)
    
    
def d2(option: EuropeanOption) -> float:
    # function to calculate the intermediate value d2 in the Black-Scholes formula
    return d1(option) - option.sigma*np.sqrt(option.tau)


def PV(option: EuropeanOption) -> float:
    # function to calculate the discount factor
    return np.exp(-option.r*option.tau)

In [22]:
euro_call = EuropeanOption(True, 100, 110, 0.25, 0.5, 0.05)
euro_put = EuropeanOption(False, 100, 110, 0.25, 0.5, 0.05)
print(Black_Scholes_Price_Call(euro_call))
print(Black_Scholes_Price_Call(euro_put))

4.22578239296007
11.509872716076657
